In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cvxpy as cp

In [4]:

def load_data():
    """Load historical data for TSLA, BND, and SPY."""
    assets = ['TSLA', 'BND', 'SPY']
    data = {}
    for asset in assets:
        file_path = os.path.join('financial_data', f'{asset}_historical_data_2015_2025.csv')
        try:
            df = pd.read_csv(file_path)
            df['Date'] = pd.to_datetime(df['Date'])
            df.set_index('Date', inplace=True)
            data[asset] = df['Daily_Return']
        except FileNotFoundError:
            print(f"Error: {file_path} not found.")
            return None
    return pd.concat(data, axis=1)

def get_forecasted_return():
    """Use ARIMA forecast from Task 2 as TSLA expected return."""
    # Simulate ARIMA forecast mean daily return (adjust based on Task 3 output)
    return 0.002  # Annualized later (252 trading days)

def calculate_historical_returns(data):
    """Calculate annualized historical average returns and covariance matrix."""
    returns = data.dropna()
    mean_returns = returns.mean() * 252  # Annualize daily returns
    cov_matrix = returns.cov() * 252    # Annualize covariance
    return mean_returns.values, cov_matrix.values  # Convert to NumPy arrays

def optimize_portfolio(mean_returns, cov_matrix):
    """Generate Efficient Frontier and key portfolios."""
    n_assets = len(mean_returns)
    weights = cp.Variable(n_assets)
    gamma = cp.Parameter(nonneg=True)
    risk = cp.quad_form(weights, cov_matrix)
    ret = mean_returns.T @ weights  # Use matrix multiplication with transposed mean_returns
    constraints = [cp.sum(weights) == 1, weights >= 0]
    
    # Objective for Efficient Frontier
    objective = cp.Maximize(ret - gamma * risk)
    problem = cp.Problem(objective, constraints)
    
    # Generate points on Efficient Frontier
    risks, returns = [], []
    gammas = np.logspace(-2, 3, 50)
    for g in gammas:
        gamma.value = g
        problem.solve()
        risks.append(cp.sqrt(risk).value)
        returns.append(ret.value)
    
    # Maximum Sharpe Ratio Portfolio
    # Approximate initial gamma for Sharpe maximization
    gamma.value = 1 / (np.mean(mean_returns) / np.mean(cov_matrix))
    problem.solve()
    max_sharpe_weights = weights.value
    max_sharpe_ret = ret.value
    max_sharpe_risk = cp.sqrt(risk).value
    
    # Minimum Volatility Portfolio
    objective_min_vol = cp.Minimize(risk)
    problem_min_vol = cp.Problem(objective_min_vol, constraints)
    problem_min_vol.solve()
    min_vol_weights = weights.value
    min_vol_ret = ret.value
    min_vol_risk = cp.sqrt(risk).value
    
    return risks, returns, max_sharpe_weights, max_sharpe_ret, max_sharpe_risk, min_vol_weights, min_vol_ret, min_vol_risk

def plot_efficient_frontier(risks, returns, max_sharpe_ret, max_sharpe_risk, min_vol_ret, min_vol_risk):
    """Plot Efficient Frontier with key portfolios."""
    sns.set_style('darkgrid')
    
    plt.figure(figsize=(10, 6))
    plt.scatter(risks, returns, c='blue', alpha=0.5, label='Efficient Frontier')
    plt.scatter(min_vol_risk, min_vol_ret, c='green', s=100, label='Minimum Volatility Portfolio')
    plt.scatter(max_sharpe_risk, max_sharpe_ret, c='red', s=100, label='Maximum Sharpe Ratio Portfolio')
    plt.xlabel('Annualized Volatility (Risk)')
    plt.ylabel('Annualized Return')
    plt.title('Efficient Frontier for TSLA, BND, SPY Portfolio')
    plt.legend()
    plt.savefig(os.path.join('portfolio_plots', 'efficient_frontier.png'))
    plt.close()

def recommend_portfolio(max_sharpe_weights, max_sharpe_ret, max_sharpe_risk, min_vol_weights, min_vol_ret, min_vol_risk, mean_returns, cov_matrix):
    """Recommend and summarize the optimal portfolio."""
    # Calculate Sharpe Ratios
    risk_free_rate = 0.02  # Assuming 2% annual risk-free rate
    max_sharpe_sharpe = (max_sharpe_ret - risk_free_rate) / max_sharpe_risk
    min_vol_sharpe = (min_vol_ret - risk_free_rate) / min_vol_risk
    
    print("\nPortfolio Optimization Results:")
    print(f"Maximum Sharpe Ratio Portfolio:")
    print(f"- Weights: TSLA={max_sharpe_weights[0]:.2%}, BND={max_sharpe_weights[1]:.2%}, SPY={max_sharpe_weights[2]:.2%}")
    print(f"- Expected Return: {max_sharpe_ret:.2%}")
    print(f"- Volatility: {max_sharpe_risk:.2%}")
    print(f"- Sharpe Ratio: {max_sharpe_sharpe:.2f}")
    print(f"Minimum Volatility Portfolio:")
    print(f"- Weights: TSLA={min_vol_weights[0]:.2%}, BND={min_vol_weights[1]:.2%}, SPY={min_vol_weights[2]:.2%}")
    print(f"- Expected Return: {min_vol_ret:.2%}")
    print(f"- Volatility: {min_vol_risk:.2%}")
    print(f"- Sharpe Ratio: {min_vol_sharpe:.2f}")
    
    # Recommendation
    if max_sharpe_sharpe > min_vol_sharpe * 1.2:  # Threshold for preferring higher risk-adjusted return
        optimal_weights = max_sharpe_weights
        opt_ret = max_sharpe_ret
        opt_risk = max_sharpe_risk
        opt_sharpe = max_sharpe_sharpe
        justification = "Recommended the Maximum Sharpe Ratio Portfolio to prioritize maximum risk-adjusted return, leveraging TSLA's forecasted growth potential."
    else:
        optimal_weights = min_vol_weights
        opt_ret = min_vol_ret
        opt_risk = min_vol_risk
        opt_sharpe = min_vol_sharpe
        justification = "Recommended the Minimum Volatility Portfolio to prioritize lower risk, balancing with BND's stability and SPY's diversification."
    
    print("\nRecommended Optimal Portfolio:")
    print(f"- Weights: TSLA={optimal_weights[0]:.2%}, BND={optimal_weights[1]:.2%}, SPY={optimal_weights[2]:.2%}")
    print(f"- Expected Annual Return: {opt_ret:.2%}")
    print(f"- Annual Volatility: {opt_risk:.2%}")
    print(f"- Sharpe Ratio: {opt_sharpe:.2f}")
    print(f"- Justification: {justification}")

def main():
    # Create output directory
    output_dir = 'portfolio_plots'
    os.makedirs(output_dir, exist_ok=True)
    
    # Load data
    data = load_data()
    if data is None:
        return
    
    # Get expected returns
    tsla_forecast_return = get_forecasted_return()
    mean_returns, cov_matrix = calculate_historical_returns(data)
    mean_returns = np.array([tsla_forecast_return * 252, mean_returns[1], mean_returns[2]])  # Replace TSLA with forecast, keep BND and SPY historical
    
    # Optimize portfolio
    risks, returns, max_sharpe_weights, max_sharpe_ret, max_sharpe_risk, min_vol_weights, min_vol_ret, min_vol_risk = optimize_portfolio(mean_returns, cov_matrix)
    
    # Plot Efficient Frontier
    plot_efficient_frontier(risks, returns, max_sharpe_ret, max_sharpe_risk, min_vol_ret, min_vol_risk)
    
    # Recommend portfolio
    recommend_portfolio(max_sharpe_weights, max_sharpe_ret, max_sharpe_risk, min_vol_weights, min_vol_ret, min_vol_risk, mean_returns, cov_matrix)

if __name__ == "__main__":
    main()


Portfolio Optimization Results:
Maximum Sharpe Ratio Portfolio:
- Weights: TSLA=100.00%, BND=0.00%, SPY=0.00%
- Expected Return: 50.40%
- Volatility: 59.46%
- Sharpe Ratio: 0.81
Minimum Volatility Portfolio:
- Weights: TSLA=0.00%, BND=95.63%, SPY=4.37%
- Expected Return: 0.12%
- Volatility: 5.47%
- Sharpe Ratio: -0.34

Recommended Optimal Portfolio:
- Weights: TSLA=100.00%, BND=0.00%, SPY=0.00%
- Expected Annual Return: 50.40%
- Annual Volatility: 59.46%
- Sharpe Ratio: 0.81
- Justification: Recommended the Maximum Sharpe Ratio Portfolio to prioritize maximum risk-adjusted return, leveraging TSLA's forecasted growth potential.
